In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Load the data from the specified table in the Excel file
df = pd.read_excel('data/REM-Data base Los Palacios.xlsx', sheet_name='Static conditions', skiprows=3)  # Adjust the sheet_name and skiprows if necessary

# Print the column names to inspect
print(df.columns)

# Assuming the correct column names are 'Date (DD/MM/YY)' and 'Corrected water table (m) 01'
date_column = 'Date (DD/MM/YY)'  
corrected_water_table_column = 'Corrected water table (m) 01' 

# Convert the date column to datetime
df[date_column] = pd.to_datetime(df[date_column], format='%d/%m/%y')

# Filter out rows where 'Corrected water table (m) 01' is NaN
df = df.dropna(subset=[corrected_water_table_column])

# Create labels for the start and end dates
start_date_label = widgets.Label(value='Start Date')
end_date_label = widgets.Label(value='End Date')

# Create labels to display the selected dates
start_date_display = widgets.Label(value='')
end_date_display = widgets.Label(value='')

# Create the slider
date_range_slider = widgets.SelectionRangeSlider(
    options=[(date.strftime('%d/%m/%y'), date) for date in df[date_column]],
    index=(0, len(df[date_column]) - 1),
    description='',
    orientation='horizontal',
    layout={'width': '500px'},
    style={'description_width': '0px'}  # Remove the description width to hide default display
)

# Add custom CSS to hide the default display of the slider values
slider_style = """
<style>
    .widget-readout {
        display: none !important;
    }
</style>
"""

# Function to update the displayed sentence based on slider values
def update_sentence(change):
    x1_date, x2_date = change['new']
    x1_value = df.loc[df[date_column] == x1_date, corrected_water_table_column]
    x2_value = df.loc[df[date_column] == x2_date, corrected_water_table_column]
    x1_display = f"{x1_value.values[0]} m" if not x1_value.empty else 'No data'
    x2_display = f"{x2_value.values[0]} m" if not x2_value.empty else 'No data'
    display_sentence.value = f"Here we can see the evolution of the Water Table in well S-1 from {x1_display} to {x2_display}."
    start_date_display.value = x1_date.strftime('%d/%m/%y')
    end_date_display.value = x2_date.strftime('%d/%m/%y')

# Display the initial sentence
x1_date, x2_date = date_range_slider.value
x1_value = df.loc[df[date_column] == x1_date, corrected_water_table_column]
x2_value = df.loc[df[date_column] == x2_date, corrected_water_table_column]
x1_display = f"{x1_value.values[0]} m" if not x1_value.empty else 'No data'
x2_display = f"{x2_value.values[0]} m" if not x2_value.empty else 'No data'
display_sentence = widgets.HTML(
    value=f"Here we can see the evolution of the Water Table in well S-1 from {x1_display} to {x2_display}."
)
start_date_display.value = x1_date.strftime('%d/%m/%y')
end_date_display.value = x2_date.strftime('%d/%m/%y')

# Attach the update function to the slider
date_range_slider.observe(update_sentence, names='value')

# Create a HBox to stack the date range title, labels, and slider
date_range_title = widgets.HTML(value="<b>Date Range</b>")
slider_box = widgets.HBox([start_date_label, start_date_display, date_range_slider, end_date_display, end_date_label])

# Display the components along with custom CSS
display(date_range_title, slider_box, display_sentence)
display(HTML(slider_style))